In [4]:
import numpy as np
import pandas as pd

def simulate_power_fluctuation_with_real_values(data, t, N):
    """
    模拟功率波动调节策略，使用真实值代替预测值
    :param data: 包含实时功率数据的 DataFrame，必须有 'power_Sum' 列
    :param t: 波动阈值
    :param N: 备用发电机数量
    :return: 修正后的数据和评估结果
    """
    # 初始化需要的变量
    k_values = []  # 用于记录修正后的 k 值
    updated_power = data['power_Sum'].copy()  # 用于更新的实时功率序列
    generator_status = 'ON'  # 初始备用发电机状态

    for current_time in range(1801, len(data) - 300):  # 确保索引不会越界
        # 获取当前及之前300秒的功率数据（实际值）
        historical_data = updated_power[current_time-300:current_time]

        # 使用真实值替代预测值
        future_real_values = updated_power[current_time:current_time+300]

        # 计算第300秒后的 k 值
        p = future_real_values.iloc[-1]  # 第300秒的真实值
        past_actual = updated_power[current_time-1500:current_time]  # 当前及过去1500秒实际值
        combined_values = np.concatenate([past_actual, future_real_values])  # 当前到未来300秒的组合值
        q = np.mean(combined_values)  # 计算 q 值
        k = (p - q) / q  # 计算 k 值

        # 保存当前发电机状态以供比较
        previous_status = generator_status

        # 调整备用发电机状态
        if k > t:
            generator_status = 'OFF'
        elif k < -t:
            generator_status = 'ON'

        # 修正下一秒功率值
        if current_time + 1 < len(data):
            real_next = updated_power.iloc[current_time + 1]  # 下一秒的真实值
            if generator_status != previous_status:  # 状态发生变化
                if previous_status == 'ON' and generator_status == 'OFF':  # 开到关
                    next_value = real_next * (1 - N / 11)
                elif previous_status == 'OFF' and generator_status == 'ON':  # 关到开
                    next_value = real_next / (1 - N / 11)
            else:  # 状态不变
                next_value = real_next

            # 更新实时功率序列
            updated_power.iloc[current_time + 1] = next_value

        # 记录修正后的 k 值
        k_values.append(k)

    # 评估策略
    k_values = np.array(k_values)
    r = np.sum((k_values >= -t) & (k_values <= t)) / len(k_values)

# 返回结果
    data['adjusted_power'] = updated_power
    return data, r

# 示例调用
data = pd.read_csv('Supplement_S1_power1_sum.csv')  # 假设文件有 'power_Sum' 列
t = 0.05  # 设置波动阈值
N = 3  # 设置备用发电机数量

# 确保数据是 DataFrame
if isinstance(data, pd.Series):
    data = data.to_frame(name='power_Sum')

adjusted_data, r = simulate_power_fluctuation_with_real_values(data, t, N)
adjusted_data.to_csv('k_adjusted.csv', index=False)
print(f"调节策略的成功率: {r:.2%}")

FileNotFoundError: [Errno 2] No such file or directory: 'Supplement_S1_power1_sum.csv'